In [115]:
pip install pyldavis

  Created wheel for pyldavis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97716 sha256=8c0cab664aecdc97618b199464e155654c35ca37b2123bf1f5c637ca95bb8ea6
  Stored in directory: c:\users\wgvan\appdata\local\pip\cache\wheels\31\8c\a0\24a443892f2134e691d59c8c6c35e19821e02f85e49871f8fd
Successfully built pyldavis



In [116]:
import pandas as pd
import os
import re
import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import spacy
import en_core_web_sm
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim
import pickle 
import pyLDAvis

In [70]:
#Read review data from csv
reviews = pd.read_csv("C:/Users/wgvan/Desktop/reviews.csv", sep=',', engine = 'python', quotechar='\"', escapechar='\\')
#drop irrelevant review columns
reviews = reviews.drop(columns=['id', 'workId', 'reviewId', 'postTime', 'episodesSeen', 'author', 'peopleFoundUseful'], axis=1)
reviews.head()

,workName,overallRating,storyRating,animationRating,soundRating,characterRating,enjoymentRating,review
0,Cowboy_Bebop,10,8,10,10,10,10,Cowboy Bebop is an episodic series. By episodi...
1,Utawarerumono,8,8,7,7,9,8,Utawarerumono manages to be one of those harem...
2,Hajime_no_Ippo,10,10,9,9,10,10,"First, let me say that I\'m not a fan of boxin..."
3,Gensoumaden_Saiyuuki,9,8,6,7,10,9,Saiyuki is one of those animes that just grabs...
4,Ranma_Â½,7,6,8,8,8,7,A comedy/romance based on the manga by Rumiko ...


In [71]:
#Read genre data from csv
genres = pd.read_csv("C:/Users/wgvan/Desktop/anime.csv", sep=',', engine = 'python', quotechar='\"', escapechar='\\')
genres.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,GintamaÂ°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [72]:
#replace some punctuation and spaces with _ so that the names in the two files are the same
genres['name'].replace('\s+', '_',regex=True,inplace=True)
genres['name'].replace(':', '_',regex=True,inplace=True)
genres['name'].replace(';', '_',regex=True,inplace=True)
genres['name'].replace('&', '_',regex=True,inplace=True)
genres['name'].replace('\'', '',regex=True,inplace=True)
genres['name'].replace('!', '',regex=True,inplace=True)
genres.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi_no_Na_wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal_Alchemist__Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,GintamaÂ°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins_Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama_#039_,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [73]:
#make a dictionary out of the two needed genre columns
genre_dict = pd.Series(genres.genre.values, index = genres.name).to_dict()

In [74]:
#add genre column to the reviews
reviews['genre'] = reviews['workName'].map(genre_dict)
reviews.head()

,workName,overallRating,storyRating,animationRating,soundRating,characterRating,enjoymentRating,review,genre
0,Cowboy_Bebop,10,8,10,10,10,10,Cowboy Bebop is an episodic series. By episodi...,"Action, Adventure, Comedy, Drama, Sci-Fi, Space"
1,Utawarerumono,8,8,7,7,9,8,Utawarerumono manages to be one of those harem...,"Action, Drama, Fantasy, Sci-Fi"
2,Hajime_no_Ippo,10,10,9,9,10,10,"First, let me say that I\'m not a fan of boxin...","Comedy, Drama, Shounen, Sports"
3,Gensoumaden_Saiyuuki,9,8,6,7,10,9,Saiyuki is one of those animes that just grabs...,"Adventure, Comedy, Drama, Shounen, Supernatural"
4,Ranma_Â½,7,6,8,8,8,7,A comedy/romance based on the manga by Rumiko ...,"Comedy, Fantasy, Martial Arts, Slice of Life"


In [78]:
print(len(reviews.index))
reviews.isnull().sum(axis=0)

135206


workName               0
overallRating          0
storyRating            0
animationRating        0
soundRating            0
characterRating        0
enjoymentRating        0
review                 2
genre              29276
dtype: int64

In [80]:
#remove punctiation and convert everything to lowercase
reviews['review_processed'] = reviews['review'].map(lambda x: re.sub('[,\.!?]', '', str(x)))
reviews['review_processed'] = reviews['review_processed'].map(lambda x: x.lower())
reviews['review_processed'].head()

0    cowboy bebop is an episodic series by episodic...
1    utawarerumono manages to be one of those harem...
2    first let me say that i\'m not a fan of boxing...
3    saiyuki is one of those animes that just grabs...
4    a comedy/romance based on the manga by rumiko ...
Name: review_processed, dtype: object

In [87]:
def toWords(sentences):
    for s in sentences:
        yield(gensim.utils.simple_preprocess(str(s), deacc=True))
        
data = reviews.review_processed.values.tolist()
data_words = list(toWords(data))

print(data_words[:1])

[['cowboy', 'bebop', 'is', 'an', 'episodic', 'series', 'by', 'episodic', 'mean', 'that', 'one', 'episode', 'doesn', 'necessarily', 'lead', 'or', 'follow', 'the', 'next', 'one', 'with', 'that', 'being', 'said', 'don', 'expect', 'lot', 'of', 'plot', 'in', 'this', 'series', 'you', 'watch', 'cowboy', 'bebop', 'for', 'the', 'characters', 'plain', 'and', 'simple', 'spike', 'faye', 'jet', 'ed', 'ein', 'are', 'what', 'make', 'this', 'anime', 'arguably', 'one', 'of', 'the', 'most', 'memorable', 'series', 'ever', 'produced', 'the', 'music', 'is', 'awesome', 'especially', 'if', 'you', 're', 'fan', 'of', 'jazz', 'the', 'animation', 'is', 'top', 'notch', 'considering', 'when', 'the', 'series', 'came', 'out', 'and', 'the', 'enjoyment', 'value', 'is', 'second', 'to', 'nonewhile', 'lot', 'of', 'people', 'want', 'that', 'similar', 'episode', 'feel', 'if', 'you', 're', 'the', 'kind', 'of', 'person', 'that', 'enjoys', 'fast', 'paced', 'anime', 'with', 'each', 'character', 'getting', 'his', 'or', 'her', '

In [88]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [93]:
#remove stopwords
stop_words = stopwords.words('english')

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [108]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model
nlp = en_core_web_sm.load()
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[['mean', 'episode', 'necessarily', 'lead', 'follow', 'next', 'say', 'expect', 'lot', 'plot', 'character', 'plain', 'simple', 'spike_faye', 'jet', 'make', 'arguably', 'memorable', 'series', 'ever', 'produce', 'music', 'awesome', 'especially', 'fan', 'jazz', 'animation', 'consider', 'come', 'enjoyment', 'value', 'second', 'nonewhile', 'lot', 'people', 'want', 'similar', 'episode', 'feel', 'kind', 'person', 'enjoy', 'fast_pace', 'anime', 'character', 'get', 'spotlight', 'type', 'enjoy', 'fast_pace', 'fun', 'rather', 'deep', 'thought_provoking', 'dark', 'mysterious', 'watch', 'cowboy_bebop', 'vicious', 'character', 'able', 'bring', 'little', 'sinister', 'side', 'kind', 'show']]


In [110]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 3), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1)]]


In [111]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [113]:
# Compute Coherence Score and perplexity
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Coherence Score:  0.37984814370674086

Perplexity:  -7.942371107671516


In [117]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x             y  topics  cluster       Freq
topic                                                    
7      0.225127  3.010518e-02       1        1  28.937664
9      0.176999 -9.745258e-07       2        1  23.599121
4      0.191664 -2.905345e-02       3        1  23.145454
8      0.129466 -3.788755e-02       4        1  12.917739
6      0.135541  2.541518e-02       5        1   4.304065
5      0.042213  1.288457e-01       6        1   1.841563
2     -0.094715 -7.397292e-02       7        1   1.785229
3     -0.258289  1.701565e-01       8        1   1.759187
0     -0.217640 -3.443131e-01       9        1   0.952768
1     -0.330365  1.307055e-01      10        1   0.757201, topic_info=         Term           Freq          Total Category  logprob  loglift
421    season   92076.000000   92076.000000  Default  30.0000  30.0000
111     first  134492.000000  134492.000000  Default  29.0000  29.0000
14    episode  201849.000000  201849.000000  Default  28.0000  28.0000
468     movie   70063.000000   70063.000000  Default  27.0000  27.0000
390      girl   86716.000000   86716.000000  Default  26.0000  26.0000
...       ...            ...            ...      ...      ...      ...
2702    rival     868.910217    3772.462402  Topic10  -5.4186   3.4151
4067     trap     848.412415    3413.419922  Topic10  -5.4424   3.4912
797      real    1168.982178   38520.367188  Topic10  -5.1219   1.3882
71     battle     860.523315   23024.160156  Topic10  -5.4283   1.5965
537      lose     804.791077   20914.677734  Topic10  -5.4952   1.6257

[791 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
19301      9  0.997752  abomination
41766      4  0.999485        abyss
60         1  0.352551       action
60         2  0.268727       action
60         3  0.022177       action
...      ...       ...          ...
1072       4  0.718630        young
1072       5  0.046636        young
1072       9  0.036174        young
23042      3  0.996844        yuuki
9796       9  0.999580       zombie

[2576 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 10, 5, 9, 7, 6, 3, 4, 1, 2])